In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GUTENBERG_CONFIG, GUTENBERG_DPO_CONFIG, SMOL_LM_135M

In [ ]:
cfg = GUTENBERG_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 800
cfg.eval_steps = 5
cfg.eval_data_mode = "fixed"
wrapper = TrainerWrapper(cfg)

In [4]:
wrapper.init_model()

2025-02-22 22:37:23.466 | INFO     | trl_wrapper.trainer_wrapper:init_model:295 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: flash_attention_2


In [7]:
wrapper.init_data_module()

2025-02-22 22:42:14.727 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:316 - Using dataset path: dataset_files/gutenberg_conversations.parquet
2025-02-22 22:42:14.729 | INFO     | model.utils:__init__:118 - Cache dir: ../dataset_caches/conversation_data_module
2025-02-22 22:42:14.729 | INFO     | model.utils:setup:159 - Loading dataset for stage fit
2025-02-22 22:42:14.730 | INFO     | model.utils:setup:162 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module, using cache: False
2025-02-22 22:42:14.731 | INFO     | model.utils:init_dataset:134 - Loading local dataset from ../dataset_files/gutenberg_conversations.parquet
2025-02-22 22:42:14.742 | INFO     | model.utils:init_dataset:144 - Selecting 800 samples
2025-02-22 22:42:14.759 | INFO     | model.utils:init_dataset:154 - Loading custom chat template: ../chat_templates/llama3.jinja
2025-02-22 22:42:14.760 | INFO     | model.utils:setup:171 - Train dataset samples: 720 Val da

DatasetConfig(batch_size=8, max_sequence_length=1512, tuning_mode='sft', using_mistral=False, notebook_mode=True, input_dataset_name='dataset_files/gutenberg_conversations.parquet', max_samples=800, custom_chat_template='llama3', train_on_inputs=False)


Map: 100%|██████████| 80/80 [00:00<00:00, 670.87 examples/s]

['input_ids', 'attention_mask', 'assistant_mask', 'labels']


In [8]:
wrapper.data_module.train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'assistant_mask', 'labels'],
    num_rows: 720
})

In [37]:
wrapper.init_trainer()

2025-02-22 22:50:29.356 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:361 - Saving output to: ./runs/02-22-22-50-390448-smollm2-135m-instruct--gutenberg-conv
2025-02-22 22:50:29.359 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:384 - Initializing trainer, run_name: 02-22-22-50-390448-smollm2-135m-instruct--gutenberg-conv, wandb project: gutenberg
/home/brianf/smolmodels/notebooks/../trl_wrapper/trainer_wrapper.py:449: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomSFTTrainer.__init__`. Use `processing_class` instead.
  self.trainer = CustomSFTTrainer(


In [17]:
wrapper.trainer.eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'assistant_mask', 'labels'],
    num_rows: 80
})

In [ ]:
wrapper.trainer.get_eval_dataloader()

Dataset({
    features: ['input_ids', 'attention_mask', 'assistant_mask', 'labels'],
    num_rows: 80
})

In [45]:
wrapper.trainer.evaluate()

2025-02-22 22:52:31.641 | INFO     | trl_wrapper.sft_trainer:evaluation_loop:94 - Generating samples...
2025-02-22 22:52:31.648 | INFO     | trl_wrapper.sft_trainer:generate_from_model:59 - Generating policy samples, max length: 1024...
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


dict_keys(['input_ids', 'attention_mask', 'assistant_mask', 'labels'])


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
2025-02-22 22:52:51.734 | INFO     | trl_wrapper.sft_trainer:evaluation_loop:119 - Generated samples.
2025-02-22 22:52:51.802 | INFO     | synthetic_data.utils:log_to_file:327 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ ref                                                │ policy                                             │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ \n<|start_header_id|>system<|end_header_id|>Cuttin │ A crowd gathered around the chateau door.**Young   │ \n<|start_header_id|>system<|end_header_id|>Cuttin │
│ g Knowledge Date: December 2023Today Date: 22 Feb  │ man**: "You know my father," said the young man    │ g Knowledge Date: December 2023Today Date: 22 Feb  │
│ 2025<|eot_id|><|start_header_id|>user<|end_header_ │ with a shrug of his shoulders.**Young man**: "He   │ 2025<|eot_id|><|start_header_id|>user<|end_header_ │
│ id|>Set in late 17th-century Fra

{'eval_loss': 3.1576321125030518,
 'eval_model_preparation_time': 0.0191,
 'eval_runtime': 24.676,
 'eval_samples_per_second': 3.242,
 'eval_steps_per_second': 1.621}

In [ ]:
wrapper.train()

2025-02-22 22:53:22.048 | INFO     | trl_wrapper.trainer_wrapper:train:604 - Starting training.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


2025-02-22 22:53:31.066 | INFO     | trl_wrapper.sft_trainer:evaluation_loop:93 - Generating samples...
2025-02-22 22:53:31.070 | INFO     | trl_wrapper.sft_trainer:generate_from_model:59 - Generating policy samples, max length: 1024...
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
